# Implementing a block

Here we will show the steps you should follow to implement a block in deeplay.

## 1. Should I implement a block?

The first step is to ensure that what you want to implement is actually a block. 
The main utility of a block is that the order of operations can be changed.
If this is useful for your module, then you should implement a block.

Another reason to implement a block is if you want users to be able to add or
remove steps from the block. For example, adding an activation function, or
removing a dropout layer. 

Also, remember that blocks shouls be small and modular. If you are implementing
a block that is too big, you should consider breaking it down into smaller blocks.

Finally, blocks should be pretty strict in terms of input and output. This is 
important to ensure that the user has the flexibility to change the order of
operations without breaking the model.

## 2. Implementing the block

The first step is to create a new file in the `deeplay/blocks` directory. It
can be in a deeper subdirectory if it makes sense.

# 2.1 The BaseBlock class

The file should contain a class that inherits from `BaseBlock`. The first 
arguments should specify the input and output shapes of the block. This is
important to ensure that the block is used correctly.

The next arguments should specify the arguments for the default layer class.
In this case, we are using `torch.nn.Conv1d`, so we should specify the kernel
size, stride, padding, etc.

Finally, the class should accept kwargs that will be passed to the super class.

In [1]:
from deeplay.blocks.base import BaseBlock
from deeplay.external.layer import Layer

import torch.nn as nn

class MyConv1dBlock(BaseBlock):
    def __init__(self, 
                 in_channels, 
                 out_channels, 
                 kernel_size=3, 
                 stride=1, 
                 padding=0, 
                 dilation=1, 
                 groups=1, 
                 bias=True,
                 order=None, # We take order here for typing purposes
                 **kwargs):
        
        # We save the input parameters
        self.in_channels = in_channels
        self.out_channels = out_channels
        self.kernel_size = kernel_size
        self.stride = stride
        self.padding = padding
        self.dilation = dilation
        self.groups = groups
        self.bias = bias

        # Create the layer
        layer = Layer(nn.Conv1d, 
                      in_channels=in_channels, 
                      out_channels=out_channels, 
                      kernel_size=kernel_size, 
                      stride=stride, 
                      padding=padding, 
                      dilation=dilation, 
                      groups=groups, 
                      bias=bias)
        
        # Send the layers and modules to the parent class
        super(MyConv1dBlock, self).__init__(order=order, layer=layer, **kwargs)



# 2.1 Annotations

It is important to add annotations to the class and methods to ensure that the
user knows what to expect. This is also useful for the IDE to provide 
autocomplete.

In [2]:
from typing import List, Optional
from torch.nn.common_types import _size_1_t

from deeplay.module import DeeplayModule

class MyConv1dBlock(BaseBlock):

    # We annotate the attributes
    in_channels: int
    out_channels: int
    kernel_size: _size_1_t
    stride: _size_1_t
    padding: _size_1_t
    dilation: _size_1_t
    groups: int
    bias: bool

    # We also annotate layer
    layer: Layer 

    def __init__(self, 
                 in_channels: int,
                 out_channels: int,
                 kernel_size: _size_1_t = 3,
                 stride: _size_1_t = 1,
                 padding: _size_1_t = 0,
                 dilation: _size_1_t = 1, 
                 groups: int = 1,
                 bias: bool = True,
                 order: Optional[List[str]] = None,
                 **kwargs: DeeplayModule) -> None:
        
        # We save the input parameters
        self.in_channels = in_channels
        self.out_channels = out_channels
        self.kernel_size = kernel_size
        self.stride = stride
        self.padding = padding
        self.dilation = dilation
        self.groups = groups
        self.bias = bias

        # Create the layer
        layer = Layer(nn.Conv1d, 
                      in_channels=in_channels, 
                      out_channels=out_channels, 
                      kernel_size=kernel_size, 
                      stride=stride, 
                      padding=padding, 
                      dilation=dilation, 
                      groups=groups, 
                      bias=bias)
        
        # Send the layers and modules to the parent class
        super().__init__(order=order, layer=layer, **kwargs)

## 2.2 Documenting the block

The next step is to document the block. This should include a description of 
the block, the input and output shapes, and the arguments that can be passed to
the block.

In [3]:
class MyConv1dBlock(BaseBlock):
    """A block for 1D convolutional operations.

    This block performs a 1D convolutional operation on the input tensor.

    Parameters
    ----------
    in_channels : int
        The number of input channels.
    out_channels : int
        The number of output channels.
    kernel_size : int
        The size of the convolutional kernel.
    stride : int
        The stride of the convolutional operation.
    padding : int
        The padding of the convolutional operation.
    dilation : int
        The dilation of the convolutional operation.
    groups : int    
        The number of groups for the convolutional operation.
    bias : bool
        Whether to include a bias term in the convolutional operation.
    order : List[str]
        The order of the layers in the block. If None, the order is
        inferred from the order of keyword arguments, with `layer`
        always being the first layer.
    **kwargs
        Additional modules to include in the block. The keys should be
        the names of the modules and the values should be the modules.
    
    Attributes
    ----------
    in_channels : int
        The number of input channels.
    out_channels : int
        The number of output channels.
    kernel_size : int
        The size of the convolutional kernel.
    stride : int
        The stride of the convolutional operation.
    padding : int
        The padding of the convolutional operation.
    dilation : int
        The dilation of the convolutional operation.
    groups : int    
        The number of groups for the convolutional operation.
    bias : bool
        Whether to include a bias term in the convolutional operation.
    order : List[str]
        The order of the layers in the block.
    layer : Layer
        The layer that performs the convolutional operation.
    
    Input
    -----
    x : torch.Tensor (batch_size, in_channels, Any)
        The input tensor to the block.
    
    Output
    ------
    y : torch.Tensor (batch_size, out_channels, Any)
        The output tensor from the block.

    Evaluation
    ----------
    See :func:`~SequentialBlock.forward` for details.

    Examples
    --------
    >>> block = MyConv1dBlock(in_channels=3, out_channels=6, kernel_size=3)
    >>> block.build()
    MyConv1dBlock(
      (layer):  Conv1d(3, 6, kernel_size=(3,), stride=(1,))
    )

    """